This notebook purpose is to generate pickle models

In [1]:
from pandas import read_excel

In [2]:
df = read_excel("../data/Sample - Superstore.xls")

In [3]:
def prepare_y(df, category):
    data = df.loc[df['Category'] == category]
    cols = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 
            'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Quantity', 'Discount', 'Profit']
    data.drop(cols, axis=1, inplace=True)
    data = data.sort_values('Order Date')
    data = data.groupby('Order Date')['Sales'].sum().reset_index()
    data = data.set_index('Order Date')
    y = data['Sales'].resample('MS').mean()
    return y

In [4]:
def gridSearch(y):
    '''
    This function searches for hyperparametes using the SARIMAX function from statsmodels, it evens searches for trend and seasonability type.
    '''
    import grid_search
    scores = grid_search.sarima_grid_search(y, 12)

    from ast import literal_eval
    order = literal_eval(scores[0][0])[0]
    seasonal_order = literal_eval(scores[0][0])[1]
    trend = literal_eval(scores[0][0])[2]
    
    return order, seasonal_order, trend

In [5]:
def fitModel(y, order, seasonal_order, trend):
    import statsmodels.api as sm
    mod = sm.tsa.statespace.SARIMAX(y,
                                    order=order,
                                    seasonal_order=seasonal_order,
                                    trend=trend,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    return results

In [6]:
def savePickle(results, category):
    import pickle
    pickle.dump(results, open(category+".pkl","wb"))
    print('Pickle file saved as '+category+".pkl")

In [7]:
import grid_search
for x in df.Category.unique():
    category = x
    y = prepare_y(df, category)
    order, seasonal_order, trend = gridSearch(y)
    results = fitModel(y, order, seasonal_order, trend)
    grid_search.test_prediction(results, y)
    savePickle(results, category)

/home/renan/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


 > Model[[(0, 0, 0), (0, 0, 0, 12), 'n']] 845.331
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'n']] 145.601
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'n']] 191.856
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'n']] 146.166
 > Model[[(0, 0, 0), (2, 0, 0, 12), 'n']] 203.185
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'c']] 307.084
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'ct']] 156.386
 > Model[[(0, 0, 0), (0, 1, 0, 12), 'c']] 149.634
 > Model[[(0, 0, 0), (1, 0, 0, 12), 't']] 186.491
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'c']] 166.087
 > Model[[(0, 0, 1), (0, 0, 0, 12), 'n']] 533.418
 > Model[[(0, 0, 1), (0, 1, 0, 12), 'n']] 148.411
 > Model[[(0, 0, 0), (1, 1, 0, 12), 't']] 212.949
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'c']] 207.907
 > Model[[(0, 0, 0), (1, 0, 0, 12), 'ct']] 176.507
 > Model[[(0, 0, 1), (1, 0, 0, 12), 'n']] 153.155
 > Model[[(0, 0, 1), (1, 1, 0, 12), 'n']] 203.120
 > Model[[(0, 0, 0), (2, 0, 0, 12), 't']] 218.754
 > Model[[(0, 0, 0), (1, 1, 0, 12), 'ct']] 233.905
 > Model[[(0, 0, 0), (0, 0, 0, 12), 'ct']] 327.

/home/renan/anaconda3/lib/python3.7/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


<Figure size 1600x800 with 4 Axes>

<Figure size 1400x700 with 1 Axes>

...

<Figure size 1400x700 with 1 Axes>

The Mean Squared Error of our forecasts is 146696.2
The Root Mean Squared Error of our forecasts is 383.01


/home/renan/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:320: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  freq=base_index.freq)


<Figure size 1400x700 with 1 Axes>

Pickle file saved as Technology.pkl
